In [1]:
import numpy as np
import pandas as pd
import json
import re

import shinra_util as util

%load_ext autoreload
%autoreload 2


In [2]:
wiki_data = util.read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

In [3]:
wiki_text = pd.DataFrame([[entry['index']['_id'], entry['title'], entry['text']] for entry in wiki_data]).rename(columns={0: '_id', 1: 'title', 2: 'text'})

## 全ての記事データをSentenceごとにsplit

In [4]:
wiki_sentences = pd.DataFrame()
for _, entry in wiki_text.iterrows():
    text = util.clean_text(entry.text)
    wiki_sentences = \
    wiki_sentences.append(pd.DataFrame({
        '_id': entry['_id']
        , 'title': entry['title']
        , 'sentence': util.text2sentence(text)
    }))

In [5]:
wiki_sentences.to_csv("../data/wikitext_split_sentence.csv", index=False)